In [5]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torch-geometric-temporal

2.0.1+cu118
11.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 30.1 MB/s eta 0:00:00
  Using cached torch_sparse-0.6.17.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_scatter-2.1.1.tar.gz (107 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 37.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=97af22abd717c2e9eefd4b19cc952aabd4cf0f8f74fed06034a1be134024c62b
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl si

In [159]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split

# GPU support
DEVICE = torch.device('cuda') # cuda
shuffle=True
batch_size = 32

In [160]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import TGCN

from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 32)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        y = F.relu(h)
        y = self.linear(y)
        return y, h

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(50)):
    cost = 0
    hidden_state = None
    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr,hidden_state)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

model.eval()
cost = 0
hidden_state = None
for time, snapshot in enumerate(test_dataset):
    y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


MSE: 0.9937


In [161]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [201]:
import os
import json
import numpy as np
from torch_geometric.data import Data

class ASLDatasetLoader:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.sign_to_label = self._create_sign_to_label_map()

    def _create_sign_to_label_map(self):
        signs = [os.path.splitext(filename)[0] for filename in os.listdir(self.directory_path)]
        return {sign: i for i, sign in enumerate(signs)}

    def _read_file_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def _augment_data(self, frame_data, rotation_range=10, translation_range=0.05, scaling_range=0.1):
        """
        Augment the frame data with random rotation, translation, and scaling.

        :param frame_data: Dictionary containing frame landmarks and deltas.
        :param rotation_range: Maximum rotation angle in degrees.
        :param translation_range: Maximum translation as a fraction of landmark range.
        :param scaling_range: Maximum scaling factor.
        :return: Augmented frame data.
        """
        landmarks = np.array(frame_data["landmarks"])
        centroid = np.mean(landmarks, axis=0)

        # Random rotation
        theta = np.radians(np.random.uniform(-rotation_range, rotation_range))
        rotation_matrix = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])
        landmarks = np.dot(landmarks - centroid, rotation_matrix) + centroid

        # Random translation
        max_translation = translation_range * (landmarks.max(axis=0) - landmarks.min(axis=0))
        translations = np.random.uniform(-max_translation, max_translation)
        landmarks += translations

        # Random scaling
        scale = np.random.uniform(1 - scaling_range, 1 + scaling_range)
        landmarks = centroid + scale * (landmarks - centroid)

        frame_data["landmarks"] = landmarks.tolist()
        return frame_data

    # AUGMENT DATA #2
    # def _augment_data(self, frame_data, rotation_range=10, translation_range=0.05, scaling_range=0.1, jitter_sigma=0.01, shear_range=0.1):
    #     """
    #     Augment the frame data with random rotation, translation, scaling, jittering, mirroring, and shearing.

    #     :param frame_data: Dictionary containing frame landmarks and deltas.
    #     :param rotation_range: Maximum rotation angle in degrees.
    #     :param translation_range: Maximum translation as a fraction of landmark range.
    #     :param scaling_range: Maximum scaling factor.
    #     :param jitter_sigma: Standard deviation for the jitter noise.
    #     :param shear_range: Maximum shear factor.
    #     :return: Augmented frame data.
    #     """
    #     landmarks = np.array(frame_data["landmarks"])
    #     centroid = np.mean(landmarks, axis=0)

    #     # Random rotation
    #     theta = np.radians(np.random.uniform(-rotation_range, rotation_range))
    #     rotation_matrix = np.array([
    #         [np.cos(theta), -np.sin(theta)],
    #         [np.sin(theta), np.cos(theta)]
    #     ])
    #     landmarks = np.dot(landmarks - centroid, rotation_matrix) + centroid

    #     # Random translation
    #     max_translation = translation_range * (landmarks.max(axis=0) - landmarks.min(axis=0))
    #     translations = np.random.uniform(-max_translation, max_translation)
    #     landmarks += translations

    #     # Random scaling
    #     scale = np.random.uniform(1 - scaling_range, 1 + scaling_range)
    #     landmarks = centroid + scale * (landmarks - centroid)

    #     # Jittering
    #     jitter = np.random.normal(0, jitter_sigma, landmarks.shape)
    #     landmarks += jitter

    #     # Mirroring (Assuming that the landmarks represent a hand and x-axis is the relevant axis for mirroring)
    #     if np.random.rand() > 0.5:
    #         landmarks[:, 0] = -landmarks[:, 0]

    #     # Random shearing
    #     shear = np.random.uniform(-shear_range, shear_range)
    #     shear_matrix = np.array([
    #         [1, shear],
    #         [0, 1]
    #     ])
    #     landmarks = np.dot(landmarks - centroid, shear_matrix) + centroid

    #     # Random Non-linear Distortion (Optional, you can uncomment if you'd like to experiment with this)
    #     # distortion = np.random.normal(0, 0.01, landmarks.shape)
    #     # landmarks += distortion * landmarks

    #     frame_data["landmarks"] = landmarks.tolist()
    #     return frame_data

    ## AUGMENT DATA 3
    # def _augment_data(self, frame_data, rotation_range=10, translation_range=0.05,
    #               scaling_range=0.1, resample_range=(0.5, 1.5), masking_prob=0.1):
    #     """
    #     Augment the frame data with random rotation, translation, scaling, and temporal alterations.

    #     :param frame_data: Dictionary containing frame landmarks and deltas.
    #     :param rotation_range: Maximum rotation angle in degrees.
    #     :param translation_range: Maximum translation as a fraction of landmark range.
    #     :param scaling_range: Maximum scaling factor.
    #     :param resample_range: Tuple of (min, max) scaling for random resampling.
    #     :param masking_prob: Probability of masking a given landmark.
    #     :return: Augmented frame data.
    #     """
    #     landmarks = np.array(frame_data["landmarks"])

    #     print("Shape of landmarks before resampling:", np.array(frame_data["landmarks"]).shape)

    #     # Random Resampling
    #     resample_factor = np.random.uniform(resample_range[0], resample_range[1])
    #     resampled_length = int(len(landmarks) * resample_factor)
    #     resampled_landmarks = np.zeros((resampled_length, landmarks.shape[1]))

    #     for dim in range(landmarks.shape[1]):
    #         resampled_landmarks[:, dim] = np.interp(
    #             np.linspace(0, len(landmarks) - 1, resampled_length),
    #             np.arange(len(landmarks)),
    #             landmarks[:, dim]
    #         )

    #     landmarks = resampled_landmarks

    #     print("Shape of landmarks after resampling:", landmarks.shape)

    #     # Check for NaN values after resampling
    #     if np.isnan(landmarks).any():
    #         print("Warning: NaN values detected after resampling.")
    #         landmarks = np.nan_to_num(landmarks)

    #     # Recalculate the deltas based on the new resampled landmarks
    #     deltas = np.diff(landmarks, axis=0)
    #     print("Shape of deltas after calculation:", deltas.shape)
    #     # Pad deltas to ensure it's the same shape as landmarks
    #     deltas = np.vstack([deltas, np.zeros((1, landmarks.shape[1]))])



    #     centroid = np.nanmean(landmarks, axis=0)

    #     # Random Masking
    #     mask = np.random.choice([True, False], size=landmarks.shape[0], p=[1-masking_prob, masking_prob])
    #     landmarks[mask] = np.nan

    #     centroid = np.nanmean(landmarks, axis=0)

    #     # Random rotation
    #     theta = np.radians(np.random.uniform(-rotation_range, rotation_range))
    #     rotation_matrix = np.array([
    #         [np.cos(theta), -np.sin(theta)],
    #         [np.sin(theta), np.cos(theta)]
    #     ])
    #     landmarks_rot = np.dot(landmarks - centroid, rotation_matrix) + centroid
    #     landmarks = np.where(np.isnan(landmarks), landmarks, landmarks_rot)

    #     # Random translation
    #     max_translation = translation_range * (np.nanmax(landmarks, axis=0) - np.nanmin(landmarks, axis=0))

    #     # Handle potential NaN or infinite values
    #     if np.isnan(max_translation).any() or np.isinf(max_translation).any():
    #         print("Warning: NaN or infinite values detected in max_translation.")
    #         max_translation = np.nan_to_num(max_translation)

    #     translations = np.random.uniform(-max_translation, max_translation)
    #     landmarks += translations

    #     # Random scaling
    #     scale = np.random.uniform(1 - scaling_range, 1 + scaling_range)
    #     landmarks_scale = centroid + scale * (landmarks - centroid)
    #     landmarks = np.where(np.isnan(landmarks), landmarks, landmarks_scale)

    #     frame_data["landmarks"] = landmarks.tolist()
    #     return frame_data

    # def _create_graph_from_frame(self, sign_name, frame_data):
    #     landmarks = frame_data["landmarks"]
    #     deltas = frame_data["deltas"]

    #     # Create edges assuming landmarks are connected in sequence.
    #     # For a more complex hand structure, this needs to be adapted.
    #     edges = [[i, i+1] for i in range(len(landmarks) - 1)]

    #     edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    #     x = torch.tensor(np.hstack((landmarks, deltas)), dtype=torch.float)
    #     y = torch.tensor([self.sign_to_label[sign_name]], dtype=torch.long)

    #     return Data(x=x, edge_index=edge_index, y=y)

    # def _create_graph_from_frame(self, sign_name, frame_data):
    #     landmarks = np.array(frame_data["landmarks"])
    #     deltas = np.array(frame_data["deltas"])

    #     # Create edges assuming landmarks are connected in sequence.
    #     # For a more complex hand structure, this needs to be adapted.
    #     edges = [[i, i+1] for i in range(len(landmarks) - 1)]

    #     # Adjust lengths for concatenation
    #     n_landmarks = len(landmarks)
    #     landmarks = landmarks[:n_landmarks-1]
    #     deltas = deltas[:n_landmarks-1]

    #     edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    #     x = torch.tensor(np.hstack((landmarks, deltas)), dtype=torch.float)
    #     y = torch.tensor([self.sign_to_label[sign_name]], dtype=torch.long)

    #     return Data(x=x, edge_index=edge_index, y=y)

    def _create_graph_from_frame(self, sign_name, frame_data):
        landmarks = np.array(frame_data["landmarks"])
        deltas = np.array(frame_data["deltas"])

        # Adjust lengths for concatenation
        n_landmarks = len(landmarks)
        landmarks = landmarks[:n_landmarks-1]
        deltas = deltas[:n_landmarks-1]

        # Create edges based on the number of available landmarks (or nodes)
        edges = [[i, i+1] for i in range(len(landmarks) - 1)]

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        x = torch.tensor(np.hstack((landmarks, deltas)), dtype=torch.float)
        y = torch.tensor([self.sign_to_label[sign_name]], dtype=torch.long)

        return Data(x=x, edge_index=edge_index, y=y)


    def get_dataset(self, augment=False):
        dataset = []

        for filename in os.listdir(self.directory_path):
            sign_name = os.path.splitext(filename)[0]
            file_path = os.path.join(self.directory_path, filename)
            sign_data = self._read_file_data(file_path)

            for frame_data in sign_data["frames"]:
                if augment:
                  frame_data = self._augment_data(frame_data)
                graph_data = self._create_graph_from_frame(sign_name, frame_data)

                dataset.append(graph_data)

        return dataset

    def number_of_classes(self):
        return len(self.sign_to_label)

In [202]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

class GraphClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_classes, dropout_rate=0.5):
        super(GraphClassifier, self).__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.bn1 = BatchNorm(128)
        self.dropout1 = torch.nn.Dropout(dropout_rate)  # Dropout after first layer
        self.conv2 = GCNConv(128, 64)
        self.bn2 = BatchNorm(64)
        self.dropout2 = torch.nn.Dropout(dropout_rate)  # Dropout after second layer
        self.fc = torch.nn.Linear(64, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GCN layer
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        #x = F.relu(x)          # Use ReLU
        x = F.leaky_relu(x)  # Use LeakyReLU instead of ReLU
        x = self.dropout1(x)

        # Second GCN layer
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        # Global pooling across nodes
        x = global_mean_pool(x, data.batch)

        # Final classification layer
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

In [207]:
from sklearn.model_selection import train_test_split

def stratified_data_split(data_list, test_size=0.2):
    # Extract labels from data list
    labels = [data.y.item() for data in data_list]

    # Use sklearn's train_test_split with stratify option
    train_data, test_data = train_test_split(data_list, test_size=test_size, stratify=labels, random_state=42)

    return train_data, test_data

def train():
    directory_path = "/content/drive/MyDrive/Colab Notebooks/DGMD E-14 Project/Datasets/ASL"
    loader = ASLDatasetLoader(directory_path)

    # Create the entire dataset without augmentation and then perform stratified split
    data_list = loader.get_dataset()
    train_dataset, test_dataset = stratified_data_split(data_list, test_size=0.2)

    # Now augment only the training dataset
    augmented_train_dataset = loader.get_dataset(augment=True)

    num_classes = loader.number_of_classes()

    train_labels = [data.y.item() for data in train_dataset]
    test_labels = [data.y.item() for data in test_dataset]

    print("Training label distribution:", Counter(train_labels))
    print("Test label distribution:", Counter(test_labels))

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GraphClassifier(num_node_features=4, num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            optimizer.step()
            total_loss += loss.item()

            # Check for NaN loss
            if np.isnan(loss.item()):
                print("Warning: NaN loss detected!")

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch}, Loss: {avg_loss}")

        scheduler.step(avg_loss)

    model.eval()
    correct = 0
    all_preds = []
    all_labels = []

    for batch in test_loader:
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch).max(dim=1)[1]
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
            correct += pred.eq(batch.y).sum().item()

    print(f"Accuracy: {correct / len(test_dataset)}")
    print("Sample predictions:", all_preds[:20])
    print("Sample true labels:", all_labels[:20])

In [208]:
train()

Training label distribution: Counter({0: 402, 12: 363, 10: 298, 18: 282, 3: 269, 5: 265, 6: 258, 17: 247, 14: 246, 15: 243, 16: 243, 9: 236, 2: 223, 8: 220, 4: 215, 7: 214, 13: 210, 19: 203, 1: 199, 11: 199})
Test label distribution: Counter({0: 100, 12: 91, 10: 75, 18: 70, 3: 67, 5: 66, 6: 65, 17: 62, 14: 62, 15: 61, 16: 60, 9: 59, 2: 56, 8: 55, 4: 54, 7: 53, 13: 52, 19: 51, 11: 50, 1: 50})


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 0, Loss: 2.747401860695851
Epoch 1, Loss: 2.4783026176162912
Epoch 2, Loss: 2.344883905935891
Epoch 3, Loss: 2.254833359507066
Epoch 4, Loss: 2.1827439069747925
Epoch 5, Loss: 2.120556626893297
Epoch 6, Loss: 2.0770003176942655
Epoch 7, Loss: 2.037682886365094
Epoch 8, Loss: 1.987620001352286
Epoch 9, Loss: 1.968819334537168
Epoch 10, Loss: 1.9221996325480788
Epoch 11, Loss: 1.9185850235480297
Epoch 12, Loss: 1.8770140318930904
Epoch 13, Loss: 1.8481386764140069
Epoch 14, Loss: 1.8230585606792304
Epoch 15, Loss: 1.8296506442601168
Epoch 16, Loss: 1.7942481335205367
Epoch 17, Loss: 1.7383391676069815
Epoch 18, Loss: 1.7407683594317376
Epoch 19, Loss: 1.7164079637467107
Epoch 20, Loss: 1.705352722843991
Epoch 21, Loss: 1.6862711725355704
Epoch 22, Loss: 1.6609583270700672
Epoch 23, Loss: 1.6625267953812322
Epoch 24, Loss: 1.653977477097813
Epoch 25, Loss: 1.6379668410820296
Epoch 26, Loss: 1.650643605220167
Epoch 27, Loss: 1.6498609773720367
Epoch 28, Loss: 1.6114877537836003
Epoch